In [ ]:
# default_exp preprocessing

# preprocessing

> API details

In [1]:
# append our Aeye package
# TODO: fix this import issue when using the package.
import sys 
sys.path.append('../Aeye')

from datasets import Flickr8k

In [2]:
#export 
from pickle import dump
from pickle import load
from pathlib import Path

from torchvision import models
from torchvision import transforms
import torch
from torch import nn
from tqdm import tqdm



In [11]:
#export

SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.SOS_token = 0
        self.EOS_token = 1
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

#     def addSentence(self, sentence):
#         for word in sentence.split(' '):
#             self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:

flickr8k_img_dir = '/home/jithin/datasets/imageCaptioning/flicker8k/Flicker8k_Dataset'
annotation_file = '/home/jithin/datasets/imageCaptioning/captions/dataset_flickr8k.json'

transform = transforms.Compose([            
 transforms.Resize(256),                    
 transforms.CenterCrop(224),        
 transforms.ToTensor(),                     
 transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])

train = Flickr8k(flickr8k_img_dir, annotation_file, split="train", transform=transform)
val = Flickr8k(flickr8k_img_dir, annotation_file, split="val", transform=transform)
test = Flickr8k(flickr8k_img_dir, annotation_file, split="test", transform=transform)

datasets = [train, val, test]

loading train complete
loading val complete
loading test complete


In [ ]:
item = 3
dataset_train[item][0]

In [ ]:
dataset_train[item][1]

In [ ]:
dataset_train[item][2]

In [ ]:
def get_img_encodings(model, dataset):
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Force to cpu due to memory issue in gpu
    # TODO: fix memory issue in gpu
    #device = torch.device('cpu')
    features = dict()
    model.eval()
    model.to(device)
    with torch.no_grad():
        for img_id, img, sentences in tqdm(dataset):
            img = torch.unsqueeze(img, 0).to(device)
            feature = model(img)
            features[img_id] = feature.cpu().numpy()
            del img,feature
    return features

In [ ]:

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x


model = models.resnet18(pretrained=False)
model.fc = Identity()

print('The Model Architecture:')
print('------------------------')
print(model)

In [ ]:
split_names = ['train', 'val', 'test']
file_name = 'flickr8k_features_'
out_dir = Path('/home/jithin/datasets/imageCaptioning/flicker8k/preprocessed/')

for i, dataset in enumerate(datasets):
    features = get_img_encodings(model, dataset)
    print('Saving in %s%s'%(file_name,split_names[i]))
    dump(features, open(out_dir/(file_name+split_names[i]), 'wb'))

In [ ]:
def get_sentence_lists(dataset, language_name):
    lang = Lang(language_name)
    sentences_list = list()

    for idx, _, sentences in tqdm(dataset):
        for sentence in sentences:
            for word in sentence:
                lang.addWord(word)

            sentences_list.append((idx, sentence))
    return sentences_list,lang

In [ ]:
# sentence_list, lang = get_sentence_lists(datasets[2], 'flickr8k')

# print(lang.n_words, len(sentence_list))

# dump(lang, open(out_dir/'lang_val_flickr8k', 'wb'))
# dump(sentence_list, open(out_dir/'sentence_list_val_flickr8k', 'wb'))

In [13]:
#export

def indexesFromSentence(lang: Lang, sentence: list):
    return [lang.word2index[word] for word in sentence]


def tensorFromSentence(lang, sentence):
    device = torch.device('cpu' if torch.cuda.is_available else 'cpu')
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorForImageCaption(features_dict, sentence_tuple, lang):
    """
    This created the input_img, imput_vec pair that is used for training
    the model.
    
    Args:
        sentence_tuple: tuple with the img idx and tokenized sentence
        feature_dict: dict that maps the img idx and the feature tensor
        lang: The language model used.
    """
    idx, sentence = sentence_tuple
    target_tensor = tensorFromSentence(lang, sentence)
    features  = features_dict[idx]
    features = torch.from_numpy(features)
    return features, target_tensor

In [ ]:
len(sentence_list)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

img, sentence = tensorImageCaption(features, lang, sentence_list[0])

In [ ]:
img.shape

In [ ]:
sentence.shape

In [5]:
#export
data_path = '/home/jithin/datasets/imageCaptioning/flicker8k/preprocessed/'
data_path = Path(data_path)

def get_preprocessed_data(split: str):
    assert split in ['train', 'val', 'test']
    
    features_fname = 'flickr8k_features_%s'%(split)
    sentences_fname = 'sentence_list_%s_flickr8k'%(split)
    lang_fname = 'lang_%s_flickr8k'%(split)
    
    features = load(open(data_path/features_fname, 'rb'))
    sentence_list = load(open(data_path/sentences_fname, 'rb'))
    lang = load(open(data_path/lang_fname, 'rb'))
    
    return features, sentence_list,lang

In [9]:
_, sentences, lang = get_preprocessed_data('tes')
lang.n_words

AssertionError: 